# Correcting Misnamed Fieldmap Scans on Flywheel

*Project: Evolution_833922* <br>
_Author: Katja Zoner_ <br>
_Updated: 05/05/2021_ <br>

Notebook to correct the misnamed dir-AP acq-dwi fieldmap scans across the Flywheel project. The dir-AP scans were mistakenly named dir-PA at the scanner. This script identifies the dir-AP scans via the `PhaseEndodingDirection` field (for nifti files) and `PhaseEncodingDirectionPositive` field (for dicom files), then corrects the acquisition labe, `ProtocolName`, and `SeriesDescription` from `fmap-EPIdist_acq-dwi_dir-PA_seq-ABCD` to `map-EPIdist_acq-dwi_dir-AP_seq-ABCD`.

**Direction is AP if:** <br>
`PhaseEndodingDirection`:           'j-'    (nifti) <br>
`PhaseEndodingDirectionPositive`:    1      (dicom)

## 0. Import packages and setup logging.

In [ ]:
import flywheel
import logging
import sys

# Set up logging
logging.basicConfig(
    stream=sys.stdout,
    level=logging.INFO,
    format='%(levelname)s: %(message)s')
logger = logging.getLogger('rename_fw_files')

## 1. Instantiate Flywheel client, get project object. 

In [ ]:
# Get client
fw = flywheel.Client()
assert fw, "Your Flywheel CLI credentials aren't set!"

# Specify project
project_label = "Evolution_833922"

# Get project object
project = fw.projects.find_first('label="{}"'.format(project_label))
assert project, "Project not found!"

## 2. Some helpful functions... 

In [59]:
def get_sessions(session_labels=None):
    ''' Return a list of session objects corresponding to the 
        optionally-specified session labels. 
        Default: return all session objects in project.

    Args:
        session_labels (list of str) : List of session labels
    
    Returns:
        sessions: list of session objects
    '''

    # Get all sessions in project
    sessions = project.sessions()

    # Filter by optionally-specified session label list
    if session_labels:
        sessions = [s for s in sessions if s.label in session_labels]

    return sessions

In [60]:
def update_acq_label(acq_id, new_label):
    '''Update the acquisition label on the project container.

    Args:
        acq_id (str) : Acquisition ID
        new_label (str) : Updated Acquisition Label

    '''
    new_acq_object = flywheel.models.Acquisition(label = new_label)
    # Modifying the acquisition label here 
    fw.modify_acquisition(acquisition_id= acq_id, body = new_acq_object)

## 3. Main Program

1. Loop through sessions
2. Loop through fmap acq-dwi acquisitions
3. Identify AP scans
4. Fix 'ProtocolName' and 'SeriesDescription' in file metadata for AP scans
5. Fix acquisition label for AP acquisitions


In [61]:
## Main Program

# Specify sessions (optional)
#session_labels = ["BRAIN RESEARCH^SATTERTHWAITE"]
sessions = get_sessions()

# New label for AP acquisitions, ProtocolName, and SeriesDescription
AP_label='fmap-EPIdist_acq-dwi_dir-AP_seq-ABCD'

#countAP=0
#countPA=0

# Loop through each session
for sess in sessions:

    # Loop through fmap acq-dwi acquisitions (dir-AP or dir-PA)
    for acq in sess.acquisitions.find('label=~fmap-EPIdist_acq-dwi_dir-.._seq-ABCD'):
        
        acq = acq.reload()  # Can't load metadata without this line!!!
        hasAP = False       # Bool to indicate whether or not to rename this acquisition

        # Loop through files in acquisition
        for file in acq.files:

            # Check metadata field to determine if scan direction is AP
            # Dicom scans have 'PhaseEncodingDirectionPositive' field instead of 'PhaseEncodingDirection' field
            try:
                isAP = (file.info['PhaseEncodingDirection'] == 'j-')
            except KeyError:
                isAP = (file.info['CSAHeader']['PhaseEncodingDirectionPositive'] == 1)
            
            # If AP scan -> correct protocol name and series description in file metadata 
            if isAP:
                #countAP+=1
                hasAP = True
                logger.info(f"Correcting metadata for scan {file.name} in session {sess.label}")
                file.update_info({'ProtocolName': AP_label, 'SeriesDescription': AP_label})
            
            # If PA scan -> update count
            #else:
                #countPA+=1
        
        # If acquisition consists of AP scans -> update acquisition label
        if hasAP:
            logger.info(f"Renaming acquisition {acq.id} in session {sess.label}")
            update_acq_label(acq.id, AP_label)


#print(countAP)
#print(countPA)


67024.2020082616491492215139788.0.0.0.nii.gz in session 11524
INFO: Renaming acquisition 5f46ca80bcc88ccfc2a1ba27 in session 11524
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2020083118481177612104167.0.0.0.dicom.zip in session 11527
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2020083118481177612104167.0.0.0.nii.gz in session 11527
INFO: Renaming acquisition 5f4d7dea3941cbadcf4689dd in session 11527
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2020090316481361477858953.0.0.0.dicom.zip in session 11528
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2020090316481361477858953.0.0.0.nii.gz in session 11528
INFO: Renaming acquisition 5f51563992f4d32fbe4b9275 in session 11528
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2020090811591279862717604.0.0.0.dicom.zip in session 11529
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2020090811591279862717604.0.0.0.nii.gz in session 11529
INFO: Re